In [0]:
import sklearn

In [0]:
help(sklearn)

In [0]:
import glob
help(glob)

In [0]:
import pickle
help(pickle)
print(pickle.__version__)

In [0]:
import PIL
help(PIL)

In [0]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing import sequence

In [0]:
help(pd)

In [0]:
help(plt)

In [0]:
help(tf)

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Our project title is Image Captioning for Chest X-ray images.
There was no publicly readymade dateset available , 
so we had to perform web scraping on the Open Access Biomedical Image Search Engine.
We downloaded the images and also the reports associated with them to get the captions.
We have used the Findings and Impression part of the X-ray report  for building the caption.
 The dataset was divided into training and testing with 90-10 split

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [0]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

filename = "token.tsv"
# load descriptions
doc = load_doc(filename)
print(doc[:300])

In [0]:
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

In [0]:
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [0]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

In [0]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')

In [0]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = 'train_images.tsv'
train = load_set(filename)
print('Dataset: %d' % len(train))

In [0]:
train

In [0]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [0]:
# Below path contains all the images
images = 'iu_xray_images/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.png')

In [0]:
img_wo_path = []
for i in img:
    i = i.replace('iu_xray_images\\','')
    img_wo_path.append(i)
    #break

In [0]:
img = img_wo_path

In [0]:
# Below file conatains the names of images to be used in train data
train_images_file = 'train_images.tsv'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))
#print(train_images)

li = []
for i in train_images:
    a,b,c = (i.partition('\t'))
    li.append(a)
train_images = li
# Create a list of all the training images with their full path names
train_img = []
#print(train_images)
for i in img: # img is list of full path names of all images
    if i in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [0]:
train_img

In [0]:
# Below file conatains the names of images to be used in test data
test_images_file = 'test_images.tsv'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))


ltest = []
for i in test_images:
    a,b,c = (i.partition('\t'))
    ltest.append(a)
test_images = ltest

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [0]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = '<start> ' + ' '.join(image_desc) + ' <end>'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

In [0]:
train_path = []
for i in train_img:
    i = 'iu_xray_images/' + i
    train_path.append(i)
    #break

In [0]:
img

In [0]:
all_path = []
for i in img:
    i = 'iu_xray_images/' + i
    all_path.append(i)
    #break

In [0]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

In [0]:
ALL_captions = []
for key, val in descriptions.items():
    for cap in val:
        ALL_captions.append(cap)


In [0]:
len(ALL_captions)

In [0]:
ALL_captions

In [0]:
sare_cap = []
for i in ALL_captions:
    i = '<start> ' +i+' <end>'
    sare_cap.append(i)

In [0]:
len(sare_cap)

In [0]:
all_train_captions

In [0]:
train_captions = all_train_captions

In [0]:
img_name_vector = train_path

In [0]:
all_captions = sare_cap

In [0]:
all_img_name_vector = all_path

In [0]:
all_captions

In [0]:
train_captions

In [0]:
# Download image files
image_folder = 'iu_xray_images/'

In [0]:
len(train_captions), len(all_captions)

# Preprocess the images using InceptionV3

Next, we  used InceptionV3 (which is pretrained on Imagenet) to classify each image. 
We extracted features from the last convolutional layer.

First, we converted the images into InceptionV3's expected format by:

- Resizing the image to 299px by 299px
- Preprocess the images using the preprocess_input method to normalize the image so that it contains pixels 
    in the range of -1 to 1, which matches the format of the images used to train InceptionV3

In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

# Initialize InceptionV3 and load the pretrained Imagenet weights
Now we will create a tf.keras model where the output layer is the last convolutional layer in the InceptionV3 architecture. The shape of the output of this layer is 8x8x2048. We use the last convolutional layer because we are using attention here. We don't perform this initialization during training because it could become a bottleneck.

In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [0]:
IMG = img

In [0]:
# Get unique images
encode_train = sorted(set(img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())

In [0]:
# Execute from here

In [0]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
# Choose the top 3000 words from the vocabulary
top_k = 3000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
tokenizer

In [0]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [0]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [0]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [0]:
# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

## Split the data into training and testing

In [0]:
# Create training and validation sets using an 80-20 split
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=0)

In [0]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

## Create a tf.data dataset for training



 Our images and captions are ready! Next, let's create a tf.data dataset to use for training our model.

In [0]:
# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 64

In [0]:
# Load the numpy files
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [0]:
dataset

## Model


* In this example, we extract the features from the lower convolutional layer of InceptionV3 giving us a vector of shape (8, 8, 2048).
* We squash that to a shape of (64, 2048).
* This vector is then passed through the CNN Encoder (which consists of a single Fully connected layer).
* The RNN (here GRU) attends over the image to predict the next word.

In [0]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 64, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 64, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [0]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [0]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [0]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoint

In [0]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [0]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

## Training


In [0]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [0]:
@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [0]:
import time

In [0]:
EPOCHS = 80

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
plt.

In [0]:

#time.time()

In [0]:
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

In [0]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [0]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [0]:
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)
Image.open(image)
print ('Real Caption:', real_caption)
print('')
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)


In [0]:
len(img_name_val)

In [0]:
img_name_val[0]

In [0]:
image_path = '2.png'
result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)
"""
Real Caption-
Normal chest x-XXXX. The cardiac silhouette and mediastinum size are within normal limits.
There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. 
There is no evidence of pneumothorax
""";

In [0]:
image_path = '2.png'
result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)
"""
Real Caption-
Normal chest x-XXXX. The cardiac silhouette and mediastinum size are within normal limits.
There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. 
There is no evidence of pneumothorax
""";

In [0]:
"""
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)
Image.open(image)
print ('Real Caption:', real_caption)
print('')
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)

"""

In [0]:
for i,j in enumerate(img_name_val):
    print(i,j)
    

In [0]:
results = dict()

In [0]:
type(results)

In [0]:
for i,j in enumerate(img_name_val):
    # i is index
    # j is image name with location
    rid = i
    image = img_name_val[rid]
    real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
    result, attention_plot = evaluate(image)
    #Image.open(image)
    
    l = []
    l.append(real_caption)
    l.append(' '.join(result))
    #print ('Real Caption:', real_caption)
    #print('')
    #print ('Prediction Caption:', ' '.join(result))
    results[j] = l
    

In [0]:
print(len(results)/2)
print(len(img_name_val))

In [0]:
with open('myfile.txt', 'w') as f:
    print(results, file=f)

In [0]:
results

In [0]:
k = results.keys()

In [0]:
abc = []

In [0]:
for i in range(len(img_name_val)):
    abc.append(i)

In [0]:
abc

In [0]:
results_new = dict()

In [0]:
for a in k:
    if a not in abc:
        results_new[a] = results[a]

In [0]:
len(results_new)

In [0]:
len(img_name_val)

In [0]:
results = results_new

In [0]:
'CXR1875_IM-0566-2001_1.png' in img_name_val

In [0]:
iu_xray_images/CXR1875_IM-0566-2001_2.png

In [0]:
results['iu_xray_images/CXR1875_IM-0566-2001.png'][0]

In [0]:
with open('myfile.txt', 'w') as f:
    print(results, file=f)

In [0]:
ALL_captions

In [0]:
all_words = []

In [0]:
for cap in ALL_captions:
    all_words.extend(cap.split(' '))

In [0]:
#all_words = set(all_words)

In [0]:
len(all_words)

In [0]:
>>> import nltk


In [0]:
>>> nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
#example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = set(stopwords.words('english')) 
  
#word_tokens = word_tokenize(example_sent) 
word_tokens = all_words 
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
filtered_sentence = [] 
  
for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  

In [0]:
len(filtered_sentence)

In [0]:
2000/193453 * 100

In [0]:
from collections import Counter 
# Pass the split_it list to instance of Counter class. 
Counter = Counter(filtered_sentence) 
  
# most_common() produces k frequently encountered 
# input values and their respective counts. 
most_occur = Counter.most_common(1000) 
  
#print(most_occur)



In [0]:
print(most_occur)

In [0]:
freq_words = []
for i in most_occur:
    #print(i[0])
    if i[1] >=1000:
        freq_words.append(i[0])

In [0]:
len(freq_words)

In [0]:
image_path = '2.png'
result, attention_plot = evaluate(image_path)
pred_cap =  ' '.join(result)
"""
Real Caption-
Normal chest x-XXXX. The cardiac silhouette and mediastinum size are within normal limits.
There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. 
There is no evidence of pneumothorax
""";

In [0]:
pred_list = pred_cap.split(' ')

In [0]:
count = len(set(pred_list) & set(freq_words))

In [0]:
count_val = dict()
for i,j in results.items():
    count_val[i] = len(set(j[1].split(' ')) & set(freq_words))
    

In [0]:
count_val

In [0]:
d = count_val
count_val_sorted = sorted(d.items(), key=lambda x: x[1], reverse=True)

In [0]:
count_val_sorted
# If its below 10 surgeon should handle it.


In [0]:
Abnormalities = ['Opacities',
'Consolidation',
'Fibrosis',
'Calcification',
'Atelectasis',
'Blunted Costophrenic Angle',
 'Cavity',
'Tracheal Shift',
 'Cardiomegaly',
 'Hilar Prominence',
 'Pleural Effusion']

In [0]:
"""
blunted costophrenic angle, cardiomegaly, cavity, consolidation, 
fibrosis, hilar enlargement, nodule, opacity and pleural effusion
"""

"""

Finding	QXR-75k
Fibrosis	0.9172
Cardiomegaly	0.9502
Consolidation	0.9301
Cavity	0.9264
Infiltration	0.8862
Hilar Prominence	0.9097
Blunted Costophrenic Angle	0.9231
Pleural Effusion	0.9682
Any Abnormality	0.8741

"""


# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)
Image.open(image)
print ('Real Caption:', real_caption)
print('')
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)

